In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras



In [ ]:
Num_cc = np.arange(0,10)
Num_ii = 100
SNR_DB = np.arange(10,22,2)
Nt = 24
Nr = 32
QAM = 16
cesit = .0
cesir = .0



s, H, r, n = load_data


E = (QAM-1)*((2*2)/2)/3
E

In [ ]:
for snr in SNR_DB:
    sreal = np.real(s[snr])
    simag = np.imag(s[snr])
    s_real = np.concatenate((sreal,simag),1)

    rreal = np.real(r[snr])
    rimag = np.imag(r[snr])
    r_real = np.concatenate((rreal,rimag),1)

    hreal = np.real(H[snr])
    himag = np.imag(H[snr])
    hup = np.concatenate((hreal,-himag),2)
    hdown = np.concatenate((himag,hreal),2)
    h_real = np.concatenate((hup,hdown),1)

    nreal = np.real(n[snr])
    nimag = np.imag(n[snr])
    n_real = np.concatenate((nreal,nimag),1)

    if snr == SNR_DB[0]:

        s_ = s_real
        H_ = h_real
        r_ = r_real
        noise = n_real

    else:

        s_ = np.concatenate((s_ , s_real),0)
        H_ = np.concatenate((H_ , h_real),0)
        r_ = np.concatenate((r_ , r_real),0)
        noise = np.concatenate((noise , n_real),0)



In [ ]:
sigmaI = (np.var(noise,1,keepdims=True)*np.eye(s_.shape[1]))

A = (np.transpose(H_,(0,2,1)) @ H_+sigmaI)
b = np.transpose(H_,(0,2,1)) @ r_

In [ ]:
L = np.tril(A)
U = np.triu(A)
D = -(A-U-L)
L_inv = np.linalg.inv(L)

In [ ]:
b_hat = L_inv@b
U_hat = L_inv@U
D_hat = L_inv@D

In [ ]:
tr = np.arange(0,s_.shape[0],2)
te = np.arange(1,s_.shape[0],2)

s_tr = s_[tr]
s_te = s_[te]

D_hat_tr = D_hat[tr]
D_hat_te = D_hat[te]

b_hat_tr = b_hat[tr]
b_hat_te = b_hat[te]


U_hat_tr = U_hat[tr]
U_hat_te = U_hat[te]


b_tr = b[tr]
b_te = b[te]

A_tr = A[tr]
A_te = A[te]

L_i_tr = L_inv[tr]
L_i_te = L_inv[te]


H_tr = H_[tr]
H_te = H_[te]


In [ ]:
class Custom_layer(keras.layers.Layer):
    def __init__(self , Nt, act):
        super(Custom_layer, self).__init__()

        self.act = act
        self.Nt = Nt
        self.W1 = self.add_weight(initializer="normal", shape=(2*Nt,1),dtype="float32", trainable=True, name = 'W1_')#此為tf的建立方法
        self.W2 = self.add_weight(initializer="normal", shape=(2*Nt,1),dtype="float32", trainable=True, name = 'W2_')#此為tf的建立方法
        self.l  = self.add_weight(initializer="normal", shape=(2*Nt,1),dtype="float32", trainable=True, name = 'l_')#此為tf的建立方法
        

    def call(self, inputs):
        x, D_h,L_i, b, A = inputs[0], inputs[1], inputs[2], inputs[3],inputs[4]
        z1 = self.W1*x   
        z2 = self.W2*L_i@(b - A@x)        

        X = z1+z2+self.l

        return self.act(X)


class Model_train(keras.Model):
    def __init__(self, model):
        super(Model_train, self).__init__()

        self.model = model

    def call(self, inputs):
        x, D_h,L_i, b, A = inputs[0], inputs[1], inputs[2], inputs[3],inputs[4]
        List = []
        for i in range(len(self.model)):
            x = self.model[i]([x, D_h,L_i, b, A])
            List.append(x)

        S = tf.concat(List,2)

        return S,x


In [ ]:
def built_net(layer,s,Nt,act):
    model_list =[]
    S_list = []
    for i in range(layer):
        S_list.append(s)
        model_list.append(Custom_layer(Nt,act))

    return np.concatenate(S_list,2), model_list


def activation(x):
    x=np.sqrt(E)*x
    alf = 3
    out = (tf.math.sigmoid(alf*x-1)+tf.math.sigmoid(alf*x+1)-1+2*tf.math.sigmoid(alf*x-2*alf)+2*tf.math.sigmoid(alf*x+2*alf)-2)/np.sqrt(E)
    return out


def lf(outd, Sd):
    layer_ = Sd.shape[2]
    Sd = tf.constant(Sd,dtype=tf.float32)


    lf =  tf.norm(outd-Sd,ord=2,axis=1)**2

    return tf.math.reduce_sum(lf*np.log(np.arange(2,layer_+2)))


In [ ]:
x= np.linspace(-2,2,100)
plt.plot(x,activation(x))
activation(.5)

In [ ]:
layer = 10

In [ ]:
S, model_list = built_net(layer,s_tr,Nt,activation)




In [ ]:
Model = Model_train(model_list)



In [ ]:
lr = .01
opt = tf.keras.optimizers.Adam(lr)

In [ ]:

for i in range(2000):


    loss_fn = lambda:lf(Model([s_tr*0,  D_hat_tr, L_i_tr, b_tr,A_tr])[0],S)
    var_list_fn = lambda: Model.trainable_weights
    opt.minimize(loss_fn, var_list_fn)

    print('ep:',i,'Loss= ',lf(Model([s_tr*0,  D_hat_tr, L_i_tr, b_tr,A_tr])[0],S).numpy())

    
      

In [ ]:
s_out = Model.predict([s_te*0,  D_hat_te, L_i_te, b_te,A_te])[1]
M = np.array([-3,-1,1,3])
sym = np.array([[0,0],[0,1],[1,0],[1,1]])
S_Out = sym[np.expand_dims(np.argmin(np.abs(s_out*np.sqrt(E)-M),2),2)]
S_True = sym[np.expand_dims(np.argmin(np.abs(s_te*np.sqrt(E)-M),2),2)]

er = S_Out != S_True
l = int(er.shape[0]/len(SNR_DB))

ber = []
for i in range(len(SNR_DB)):
    be = np.sum(er[i*l:(i+1)*l])/l/(4*Nt)

    ber.append(be)
ber

In [ ]:

plt.semilogy(SNR_DB,ber)